In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from tqdm import tqdm

In [2]:
%%time
manhattan = gpd.read_file('D:\大论文\myCode\ArcGISPro\predict_map\Area_64.shp')
for i in tqdm(range(1, 32)):
    data = pd.read_csv(f'D:\大论文\myCode\data\day_data\day_{i}',
                        names=[
                            'pickup_datetime', 'dropoff_datetime',
                            'pickup_longitude', 'pickup_latitude',
                            'dropoff_longitude', 'dropoff_latitude', 'O', 'D',
                            'duration', 'weekday', 'day', 'hour'
                        ])
    data = data.sort_values(['pickup_datetime', 'dropoff_datetime'])
    O = gpd.GeoDataFrame(data)
    # 转为点文件
    O['geometry'] = O.apply(lambda x: Point(x['pickup_longitude'], x['pickup_latitude']), axis=1)
    # 空间连接
    O_sjoin = gpd.sjoin(O, manhattan, op='intersects')
    O_sjoin['pickup_datetime'] = pd.to_datetime(O_sjoin['pickup_datetime'])
    df = O_sjoin.pivot_table('hour', 'pickup_datetime', 'index_right', aggfunc='count', fill_value=0)
    use_data = df.resample('5T').sum()
    use_data.to_csv(f'D:\大论文\myCode\data\predict_data\predict_5T_{i}.csv')
    print(f'{i}: {len(use_data)}')

100%|██████████| 31/31 [27:35<00:00, 53.39s/it]31: 288
Wall time: 27min 35s



## 一月的数据合并

In [3]:
data_month = pd.read_csv('D:\大论文\myCode\data\predict_data\predict_5T_1.csv')
for i in range(2, 32):
    data = pd.read_csv(f'D:\大论文\myCode\data\predict_data\predict_5T_{i}.csv')
    data_month = pd.concat([data_month, data])

In [4]:
data_month['pickup_datetime'] = pd.to_datetime(data_month['pickup_datetime'])

In [5]:
data_month['day'] = data_month['pickup_datetime'].dt.day
data_month['weekday'] = data_month['pickup_datetime'].dt.weekday
data_month['hour'] = data_month['pickup_datetime'].dt.hour

In [24]:
data_month = data_month.fillna(0)

In [25]:
# 数据转矩阵并保存

In [26]:
data_demand = np.array(data_month.T.iloc[1:-3])[:, np.newaxis, :]

In [27]:
date_features = np.expand_dims(np.array(data_month.T.iloc[-3:]), 0).repeat(64, axis=0)

In [28]:
data_4inputs = np.concatenate((data_demand, date_features), axis=1)

In [29]:
np.save(r'C:\Users\Administrator\Desktop\st_model\STGCN-PyTorch-master\data\manhattan_node_values_1d.npy', data_demand)

In [30]:
np.save(r'C:\Users\Administrator\Desktop\st_model\STGCN-PyTorch-master\data\manhattan_node_values_4d.npy', data_4inputs)

## 面邻域转邻接矩阵

In [2]:
area_neighbor = pd.read_excel('D:\大论文\myCode\ArcGISPro\predict_map\polygon_neighbor.xlsx')

In [3]:
# 由于面35在GIS中是孤立的，因此其没有领域面，实际上该面的邻域认为是25和24
set([i for i in range(64)]) - set(area_neighbor.src_FID)

{35}

In [5]:
edge = area_neighbor[['src_FID', 'nbr_FID']]

In [12]:
edge = edge.append([{'src_FID':i, 'nbr_FID':j} for i,j in zip([35,35,25,24], [25,24,35,35])], ignore_index=True)

In [13]:
edge.to_csv(r'C:\Users\Administrator\Desktop\edge.csv', index=False)

In [ ]:
# 邻接矩阵

In [7]:
area_neighbor

,OBJECTID,src_FID,nbr_FID,LENGTH,NODE_COUNT
0,1,0,13,0.009848,0
1,2,0,32,0.000000,1
2,3,0,46,0.006994,0
3,4,0,50,0.010740,0
4,5,1,2,0.001169,0
...,...,...,...,...,...
317,318,63,12,0.005003,0
318,319,63,27,0.000000,1
319,320,63,28,0.005002,0
320,321,63,53,0.013506,0


In [9]:
adj_mat = np.zeros((64, 64))

In [10]:
for i, j in zip(area_neighbor.src_FID, area_neighbor.nbr_FID):
    adj_mat[i][j] = 1
for i, j in zip([35, 35, 25, 24], [25, 24, 35, 35]):
    adj_mat[i][j] = 1

In [14]:
# np.save('manhattan_adj_mat.npy', adj_mat)
np.save(r'C:\Users\Administrator\Desktop\st_model\STGCN-PyTorch-master\data\manhattan_adj_mat.npy', adj_mat)

In [5]:
# X = np.load(r'C:\Users\Administrator\Desktop\st_model\mydata\manhattan_node_values_1d.npy', allow_pickle=True)

In [16]:
# X.shape